# Mount google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Split dataset into subsets that MFA can handle

Create a new folder in your FastTranscribeAlign folder (in MyDrive) and name it "wav_tg_comb"

Copy all files in your wav folder and all files in your tg_raw folder into the newly created wav_tg_comb folder

In [ ]:
import os
from shutil import move

# Set your source directory
source_directory = '/content/drive/MyDrive/FastTranscribeAlign/wav_tg_comb'
files_per_subfolder = 100  # Adjust the number of pairs you want per subfolder

# List all files and filter out non '.wav' and '.textgrid' files
all_files = os.listdir(source_directory)
wav_files = sorted([f for f in all_files if f.endswith('.wav')])
textgrid_files = sorted([f for f in all_files if f.endswith('.TextGrid')])

# Combine the files into pairs based on matching base filenames
pairs = []
for wav_file in wav_files:
    base_name = os.path.splitext(wav_file)[0]
    textgrid_file = base_name + '.TextGrid'
    if textgrid_file in textgrid_files:
        pairs.append((wav_file, textgrid_file))

# Function to create subfolders and move files
def create_subfolders_and_move_files(pairs, source_directory, files_per_subfolder):
    subfolder_count = 1
    for i in range(0, len(pairs), files_per_subfolder):
        # Create a new subfolder in the source directory
        subfolder_path = os.path.join(source_directory, f'subfolder_{subfolder_count}')
        os.makedirs(subfolder_path, exist_ok=True)

        # Move a batch of pairs to the new subfolder
        for wav_file, textgrid_file in pairs[i:i + files_per_subfolder]:
            move(os.path.join(source_directory, wav_file), os.path.join(subfolder_path, wav_file))
            move(os.path.join(source_directory, textgrid_file), os.path.join(subfolder_path, textgrid_file))

        subfolder_count += 1

    print("Files have been successfully divided into subfolders.")

# Call the function
create_subfolders_and_move_files(pairs, source_directory, files_per_subfolder)

Now your wav_tg_comb folder should contain subsets of your data. This serves as the input to MFA.

Run MFA.

# Combine MFA results into one folder

Create a new folder in your FastTranscribeAlign folder (in MyDrive) and name it "MFA_output"

Create a new folder in your FastTranscribeAlign folder (in MyDrive) and name it "MFA_output_comb"

Upload your MFA results subfolders into the newly created MFA_output folder.

In [ ]:
import os
from shutil import copy2

# Set the path to your parent directory containing all the subfolders
parent_directory = '/content/drive/MyDrive/FastTranscribeAlign/MFA_output'

# Set the path to the destination directory where you want to copy all TextGrid files
destination_directory = '/content/drive/MyDrive/FastTranscribeAlign/MFA_output_comb'
os.makedirs(destination_directory, exist_ok=True)  # Create the destination directory if it doesn't exist

def copy_textgrid_files(source_dir, dest_dir):
    for root, dirs, files in os.walk(source_dir):
        for file in files:
            if file.endswith('.TextGrid'):
                source_file_path = os.path.join(root, file)
                destination_file_path = os.path.join(dest_dir, file)

                # Check if a file with the same name already exists in the destination directory
                if os.path.exists(destination_file_path):
                    # If so, create a new file name to avoid overwriting
                    base, extension = os.path.splitext(file)
                    counter = 1
                    new_file_name = f"{base}_{counter}{extension}"
                    destination_file_path = os.path.join(dest_dir, new_file_name)

                    # Increment the counter until a unique name is found
                    while os.path.exists(destination_file_path):
                        counter += 1
                        new_file_name = f"{base}_{counter}{extension}"
                        destination_file_path = os.path.join(dest_dir, new_file_name)

                # Copy the file
                copy2(source_file_path, destination_file_path)

    print("All TextGrid files have been copied to the destination directory.")

# Call the function
copy_textgrid_files(parent_directory, destination_directory)